<h1> Classification with a Neural Network using Keras (Sequential API)</h1>

<h2>1. Imports and load data</h2>

In [1]:
import pandas as pd
import numpy as np
from typing import List, Dict
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import classification_report, accuracy_score
from aeon.classification.deep_learning import TimeCNNClassifier
from tensorflow.keras.callbacks import EarlyStopping

In [2]:
class DataProcessor:
    def __init__(self, input_path: str, file_names: List[str]) -> None:
        """
        Initializes the DataProcessor with the given input path and file names.

        Args:
            input_path (str): The directory path where the files are stored.
            file_names (List[str]): List of file names to be read.

        Returns:
            None
        """
        self.input_path = input_path
        self.file_names = file_names
        
    def read_files(self) -> Dict[str, pd.DataFrame]:
        """
        Reads the files specified in the file names and stores them in a dictionary.

        Returns:
            Dict[str, pd.DataFrame]: A dictionary where keys are file names and values are DataFrames.
        """
        self.data = {}
        print("Reading files...")
        for file in self.file_names:
            with open(self.input_path + file + '.txt', 'r') as f:
                self.data[file] = pd.read_csv(f, header=None, sep='\t')
        return self.data
    
    def print_shape(self) -> None:
        """
        Prints the shape of each loaded DataFrame.

        Returns:
            None
        """
        print("Files read:")
        for file in self.data:
            print(f"{file}: {self.data[file].shape}")
            
    def create_target_df(self) -> pd.Series:
        """
        Renames the columns in the data['target'] and creates the wanted target DataFrame by extracting the column 'Valve_Condition'.

        Returns:
            pd.Series: A pandas Series containing the 'Valve_Condition' column from the target DataFrame.
        """
        target_columns = ['Cooler_Condition', 'Valve_Condition', 
                          'Internal_Pump_Leakage', 'Hydraulic_Accumulator', 
                          'Stable_Flag']
        self.data['target'].columns = target_columns
        self.valve_condition = self.data['target']['Valve_Condition']
        return self.valve_condition

def process_data() -> (Dict[str, pd.DataFrame], pd.Series): # type: ignore
    """
    Processes the data by reading the files and extracting the target DataFrame.

    Returns:
        Tuple[Dict[str, pd.DataFrame], pd.Series]: A tuple containing the data dictionary and the valve condition Series.
    """
    input_path = "input_data/"
    file_names = [
        "ce", "cp", "eps1", "se", "vs1", 
        "fs1", "fs2", 
        "ps1", "ps2", "ps3", "ps4", "ps5", "ps6",
        "ts1", "ts2", "ts3", "ts4", "target"
    ]
    
    processor = DataProcessor(input_path, file_names)
    data = processor.read_files()
    processor.print_shape()
    df_target = processor.create_target_df()
    return data, df_target

data, df_target = process_data()

Reading files...
Files read:
ce: (2205, 60)
cp: (2205, 60)
eps1: (2205, 6000)
se: (2205, 60)
vs1: (2205, 60)
fs1: (2205, 600)
fs2: (2205, 600)
ps1: (2205, 6000)
ps2: (2205, 6000)
ps3: (2205, 6000)
ps4: (2205, 6000)
ps5: (2205, 6000)
ps6: (2205, 6000)
ts1: (2205, 60)
ts2: (2205, 60)
ts3: (2205, 60)
ts4: (2205, 60)
target: (2205, 5)


<h2>2. Create input and target data </h2>

We use the six sensors which we identified as relevant during data exploration: 'eps1', 'se', 'fs1', 'ps1', 'ps2', 'ps3' 

In [3]:
df_list = ['eps1', 'se', 'fs1', 'ps1', 'ps2', 'ps3']
input_df = pd.concat([data[i] for i in df_list], axis = 1)
input_df

,0,1,2,3,4,5,6,7,8,9,...,5990,5991,5992,5993,5994,5995,5996,5997,5998,5999
0,2411.6,2411.6,2411.6,2411.6,2411.6,2411.6,2411.6,2411.6,2411.6,2409.6,...,2.336,2.391,2.375,2.297,2.328,2.383,2.328,2.250,2.250,2.211
1,2409.6,2409.6,2409.6,2409.6,2409.6,2409.6,2409.6,2409.6,2409.6,2409.6,...,2.297,2.266,2.266,2.219,2.211,2.266,2.273,2.211,2.195,2.219
2,2397.8,2397.8,2397.8,2397.8,2397.8,2397.8,2397.8,2397.8,2397.8,2395.8,...,2.359,2.391,2.391,2.375,2.375,2.375,2.305,2.305,2.320,2.266
3,2383.8,2383.8,2383.8,2383.8,2383.8,2383.8,2383.8,2383.8,2382.8,2382.8,...,2.117,2.219,2.281,2.227,2.164,2.164,2.219,2.250,2.273,2.273
4,2372.0,2372.0,2372.0,2372.0,2372.0,2372.0,2372.0,2372.0,2372.0,2373.0,...,2.141,2.172,2.187,2.227,2.219,2.211,2.242,2.219,2.227,2.297
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2200,2416.4,2416.4,2416.4,2416.4,2416.4,2416.4,2416.4,2416.4,2416.4,2416.4,...,2.328,2.305,2.328,2.359,2.375,2.281,2.242,2.250,2.266,2.273
2201,2415.6,2415.6,2415.6,2415.6,2415.6,2415.6,2415.6,2415.6,2415.6,2415.6,...,2.273,2.383,2.359,2.297,2.297,2.336,2.406,2.461,2.461,2.406
2202,2413.6,2413.6,2413.6,2413.6,2413.6,2413.6,2413.6,2413.6,2413.6,2413.6,...,2.227,2.242,2.219,2.211,2.273,2.273,2.250,2.219,2.219,2.250
2203,2413.6,2413.6,2413.6,2413.6,2413.6,2413.6,2413.6,2413.6,2413.6,2413.6,...,2.328,2.328,2.328,2.281,2.266,2.305,2.281,2.250,2.242,2.281


Standardise the input and target data

In [4]:
# Standardise the target labels
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(df_target)

# Standatdise the input
scaler = StandardScaler()
input_data_scaled = scaler.fit_transform(input_df)

<h2>3. Create the model, train it & make predictions </h2>

In [5]:
states = [27, 6728, 49122]
accs = []

In [6]:
for RANDOM_STATE in states:

    X_train, X_temp, y_train, y_temp = train_test_split(input_data_scaled, y_encoded, test_size=0.2, random_state= RANDOM_STATE, stratify=y_encoded)
    X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=RANDOM_STATE, stratify=y_temp)

    # create a callback for early stopping using keras.callbacks
    callback = EarlyStopping(
        monitor="accuracy",
        patience=5,
        verbose=1,
        mode="auto",
    )

    cnn = TimeCNNClassifier(
        n_layers = 3,
        n_filters=3,
        avg_pool_size = 2,
        n_epochs=150, 
        batch_size=32,
        loss = "categorical_crossentropy",
        verbose = True,
        callbacks = [callback],
        random_state = RANDOM_STATE
    )

    cnn.fit(X_train, y_train)

    y_pred_prob = cnn.predict_proba(X_test)
    y_pred = np.argmax(y_pred_prob, axis=1)

    print(f"Classification Report for random state {RANDOM_STATE}:")
    print(classification_report(y_test, y_pred))
    
    train_accuracy = cnn.score(X_train, y_train)
    val_accuracy = cnn.score(X_val, y_val)  # use the validation set
    test_accuracy = cnn.score(X_test, y_test)
    
    print(f"Training Accuracy: {train_accuracy}")
    print(f"Validation Accuracy: {val_accuracy}")
    print(f"Test Accuracy: {test_accuracy}")

    # calculate accuracy and append to the list for later mean and std calculation
    accuracy = accuracy_score(y_test, y_pred)
    accs.append(accuracy)
    
# calculate mean and std of accuracy scores
accs_mean = round(np.mean(accs), 4)
accs_std = round(np.std(accs), 4)

print(f"Mean Accuracy: {accs_mean}")
print(f"Std Accuracy: {accs_std}")

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 24660, 1)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d (Conv1D)                 │ (None, 24654, 3)       │            24 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ average_pooling1d               │ (None, 12327, 3)       │             0 │
│ (AveragePooling1D)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_1 (Conv1D)               │ (None, 12321, 3)       │            66 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ average_pooling1d_1             │ (None, 6160, 3)        │             0 │
│ (AveragePooling1D)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_2 (Conv1D)               │ (None, 6154, 3)        │            66 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ average_pooling1d_2             │ (None, 3077, 3)        │             0 │
│ (AveragePooling1D)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 9231)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 4)              │        36,928 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 37,084 (144.86 KB)

 Trainable params: 37,084 (144.86 KB)

 Non-trainable params: 0 (0.00 B)

...

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_2 (InputLayer)      │ (None, 24660, 1)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_6 (Conv1D)               │ (None, 24654, 3)       │            24 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ average_pooling1d_6             │ (None, 12327, 3)       │             0 │
│ (AveragePooling1D)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_7 (Conv1D)               │ (None, 12321, 3)       │            66 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ average_pooling1d_7             │ (None, 6160, 3)        │             0 │
│ (AveragePooling1D)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_8 (Conv1D)               │ (None, 6154, 3)        │            66 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ average_pooling1d_8             │ (None, 3077, 3)        │             0 │
│ (AveragePooling1D)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_2 (Flatten)             │ (None, 9231)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 4)              │        36,928 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 37,084 (144.86 KB)

 Trainable params: 37,084 (144.86 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/150
56/56 ━━━━━━━━━━━━━━━━━━━━ 2s 24ms/step - accuracy: 0.3973 - loss: 1.8414
Epoch 2/150
56/56 ━━━━━━━━━━━━━━━━━━━━ 1s 23ms/step - accuracy: 0.4891 - loss: 1.2965
Epoch 3/150
56/56 ━━━━━━━━━━━━━━━━━━━━ 1s 24ms/step - accuracy: 0.5209 - loss: 1.2738
Epoch 4/150
56/56 ━━━━━━━━━━━━━━━━━━━━ 1s 23ms/step - accuracy: 0.5275 - loss: 1.2281
Epoch 5/150
56/56 ━━━━━━━━━━━━━━━━━━━━ 1s 23ms/step - accuracy: 0.5330 - loss: 1.1468
Epoch 6/150
56/56 ━━━━━━━━━━━━━━━━━━━━ 1s 23ms/step - accuracy: 0.6192 - loss: 0.9646
Epoch 7/150
56/56 ━━━━━━━━━━━━━━━━━━━━ 1s 23ms/step - accuracy: 0.7014 - loss: 0.7658
Epoch 8/150
56/56 ━━━━━━━━━━━━━━━━━━━━ 1s 23ms/step - accuracy: 0.7846 - loss: 0.6177
Epoch 9/150
56/56 ━━━━━━━━━━━━━━━━━━━━ 1s 24ms/step - accuracy: 0.8219 - loss: 0.5131
Epoch 10/150
56/56 ━━━━━━━━━━━━━━━━━━━━ 1s 23ms/step - accuracy: 0.8759 - loss: 0.4243
Epoch 11/150
56/56 ━━━━━━━━━━━━━━━━━━━━ 1s 24ms/step - accuracy: 0.9119 - loss: 0.3459
Epoch 12/150
56/56 ━━━━━━━━━━━━━━━━━━━━ 1s 23ms/step